In [1]:
import sys
import os
sys.path.append(os.getcwd()[:-8])

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import plotly.io as pio
pio.renderers.default = 'colab'

In [4]:
from ACME.ACME import ACME
import numpy as np
import pandas as pd

In [5]:
mean = [5,5,5,5,5,5]
cov = np.identity(6)
x_normal = pd.DataFrame(np.random.multivariate_normal(mean, cov, 5000), columns = ['f'+str(i) for i in range(1,7)] )
x_normal['is_out'] = 0

In [6]:
mean = [10,10,8,5,5,5]
cov = np.identity(6)
x_outliers =  pd.DataFrame(np.random.multivariate_normal(mean, cov, 200), columns = ['f'+str(i) for i in range(1,7)] )
x_outliers['is_out'] = 1

In [7]:
X = pd.concat([x_normal,x_outliers],ignore_index=True, sort=False).reset_index(drop=True)

In [8]:
features = ['f'+str(i) for i in range(1,7)]

In [9]:
from sklearn.ensemble import IsolationForest
if_model = IsolationForest(random_state=0).fit(X[features])

In [10]:
X['pred_AD_score'] = if_model.decision_function(X[features])
X['pred_AD_score'] = -1*X['pred_AD_score']
X.tail()

,f1,f2,f3,f4,f5,f6,is_out,pred_AD_score
5195,8.644865,10.105572,9.301959,5.517671,5.513136,3.488996,1,0.153407
5196,9.646700,10.424622,7.840204,4.322793,5.584589,4.059455,1,0.129614
5197,10.305462,9.112375,8.249572,4.174266,3.478737,6.088903,1,0.138577
5198,8.989050,10.137188,7.086822,4.501622,6.219614,4.967873,1,0.097013
5199,10.180959,8.541190,7.812347,2.824749,6.058867,3.467689,1,0.154445


In [11]:
X['pred_AD_score'].plot(kind='hist')

<AxesSubplot: ylabel='Frequency'>

In [12]:
bottom_anomalies = X.loc[(X['pred_AD_score']>0.01)].sort_values(by='pred_AD_score', ascending=False).tail(10)
top_anomalies = X.loc[(X['pred_AD_score']>0.01)].sort_values(by='pred_AD_score', ascending=False).head(10)

In [13]:
top_normal = X.loc[(X['pred_AD_score']<0.01)].sort_values(by='pred_AD_score', ascending=False).tail(10)
bottom_normal = X.loc[(X['pred_AD_score']<0.01)].sort_values(by='pred_AD_score', ascending=False).head(10)

In [14]:
top_anomalies

,f1,f2,f3,f4,f5,f6,is_out,pred_AD_score
5138,10.662410,10.653745,9.562558,5.724044,3.362165,7.664553,1,0.213573
5126,11.011428,11.464443,9.225154,6.314234,6.190391,4.567922,1,0.208105
5100,12.199827,11.478920,9.135335,4.745199,3.316243,5.052440,1,0.207252
5147,11.650554,12.069350,8.393920,3.210423,4.776478,3.340703,1,0.202273
5155,12.103317,10.224171,9.472594,5.045652,5.219780,2.902010,1,0.195059
5074,10.775616,8.511565,9.668860,4.148746,4.386730,1.944638,1,0.191806
5140,10.697529,11.472186,9.085546,4.566536,2.814800,4.932826,1,0.190127
5098,10.810706,12.151990,8.036632,3.703082,6.013446,3.253574,1,0.189514
5127,11.367366,9.819782,9.957951,2.499494,5.872226,4.366396,1,0.187559
5118,9.720013,10.794377,8.942072,6.479189,6.314693,4.170145,1,0.186839


In [15]:
k = 20
def score_function(model, data):
    try: # for global
        df = model.decision_function(data)
    except: # for local
        df = model.decision_function(data.reshape(1,-1))
    
    return -1*df

acme_ifo = ACME(model = if_model, target = 'AD_score', K=k, task='ad', score_function=score_function, features=features)
acme_ifo = acme_ifo.explain(X[features], robust = True)

In [16]:
acme_local = acme_ifo.explain_local(X[features].loc[bottom_anomalies.index[-1]])

In [17]:
acme_local.summary_table(local=True)

,quantile,original,type_feature,importance,effect,predict,baseline_prediction,baseline_quantile,class
feature,,,,,,,,,
f1,0.386731,4.757300,numeric,0.022675,0.010092,0.010092,0.010092,0.386731,None
f1,0.050000,3.390467,numeric,0.022675,0.036332,0.036332,0.010092,0.386731,None
f1,0.100000,3.757871,numeric,0.022675,0.030263,0.030263,0.010092,0.386731,None
f1,0.150000,4.003165,numeric,0.022675,0.021278,0.021278,0.010092,0.386731,None
f1,0.200000,4.194759,numeric,0.022675,0.017670,0.017670,0.010092,0.386731,None
...,...,...,...,...,...,...,...,...,...
f6,0.750000,5.649117,numeric,0.020020,-0.058117,-0.058117,0.010092,0.999423,None
f6,0.800000,5.836831,numeric,0.020020,-0.049212,-0.049212,0.010092,0.999423,None
f6,0.850000,6.020082,numeric,0.020020,-0.045186,-0.045186,0.010092,0.999423,None


In [18]:
acme_local.feature_importance()

,importance
feature,
f2,0.030934
f3,0.027898
f1,0.022675
f4,0.020766
f6,0.020020
f5,0.017462


In [19]:
acme_local.feature_importance(local=True, weights = {'ratio':0.1,'delta':0.1,'change':0.3,'distance':0.5})

,ratio,delta,change,distance_to_change,max_score,min_score,local_score,importance
f6,1.000000,0.072791,1.0,0.049423,0.010092,-0.062699,0.010092,0.882568
f5,0.745608,0.038069,1.0,0.107500,0.019776,-0.018293,0.010092,0.824618
f3,0.421756,0.054443,1.0,0.110000,0.041574,-0.012870,0.010092,0.792620
f2,0.456263,0.037810,1.0,0.133654,0.030650,-0.007159,0.010092,0.782580
f4,0.091499,0.042035,0.0,1.000000,0.048281,0.006246,0.010092,0.013353
f1,0.059667,0.054039,0.0,1.000000,0.060906,0.006868,0.010092,0.011371


In [20]:
acme_local.feature_importance(local=True, weights = {'ratio':0.4,'delta':0.1,'change':0.3,'distance':0.2})

,ratio,delta,change,distance_to_change,max_score,min_score,local_score,importance
f6,1.000000,0.072791,1.0,0.049423,0.010092,-0.062699,0.010092,0.897394
f5,0.745608,0.038069,1.0,0.107500,0.019776,-0.018293,0.010092,0.780550
f2,0.456263,0.037810,1.0,0.133654,0.030650,-0.007159,0.010092,0.659555
f3,0.421756,0.054443,1.0,0.110000,0.041574,-0.012870,0.010092,0.652147
f4,0.091499,0.042035,0.0,1.000000,0.048281,0.006246,0.010092,0.040803
f1,0.059667,0.054039,0.0,1.000000,0.060906,0.006868,0.010092,0.029271


In [21]:
acme_local.feature_exploration('f1',local=True)

,quantile,original,type_feature,importance,effect,predict,baseline_prediction,baseline_quantile,class,size,direction
feature,,,,,,,,,,,
f1,0.386731,4.757300,numeric,0.022675,0.000000,0.010092,0.010092,0.386731,None,0.30,anomalies
f1,0.050000,3.390467,numeric,0.022675,0.026240,0.036332,0.010092,0.386731,None,0.05,anomalies
f1,0.100000,3.757871,numeric,0.022675,0.020171,0.030263,0.010092,0.386731,None,0.05,anomalies
f1,0.150000,4.003165,numeric,0.022675,0.011186,0.021278,0.010092,0.386731,None,0.05,anomalies
f1,0.200000,4.194759,numeric,0.022675,0.007578,0.017670,0.010092,0.386731,None,0.05,anomalies
f1,0.250000,4.363252,numeric,0.022675,0.002258,0.012350,0.010092,0.386731,None,0.05,anomalies
f1,0.300000,4.520879,numeric,0.022675,0.001732,0.011824,0.010092,0.386731,None,0.05,anomalies
f1,0.350000,4.654870,numeric,0.022675,0.000515,0.010607,0.010092,0.386731,None,0.05,anomalies
f1,0.400000,4.794466,numeric,0.022675,-0.000690,0.009402,0.010092,0.386731,None,0.05,anomalies


In [22]:
acme_local.feature_exploration('f2',local=True,plot=True)

In [23]:
acme_local.baseline_values()

,f1,f2,f3,f4,f5,f6
0,5.189268,5.191856,5.105862,4.991795,5.008653,4.994838


In [24]:
acme_local.summary_plot(local=True)

In [25]:
acme_local.summary_table(local=True)

,quantile,original,type_feature,importance,effect,predict,baseline_prediction,baseline_quantile,class
feature,,,,,,,,,
f1,0.386731,4.757300,numeric,0.022675,0.010092,0.010092,0.010092,0.386731,None
f1,0.050000,3.390467,numeric,0.022675,0.036332,0.036332,0.010092,0.386731,None
f1,0.100000,3.757871,numeric,0.022675,0.030263,0.030263,0.010092,0.386731,None
f1,0.150000,4.003165,numeric,0.022675,0.021278,0.021278,0.010092,0.386731,None
f1,0.200000,4.194759,numeric,0.022675,0.017670,0.017670,0.010092,0.386731,None
...,...,...,...,...,...,...,...,...,...
f6,0.750000,5.649117,numeric,0.020020,-0.058117,-0.058117,0.010092,0.999423,None
f6,0.800000,5.836831,numeric,0.020020,-0.049212,-0.049212,0.010092,0.999423,None
f6,0.850000,6.020082,numeric,0.020020,-0.045186,-0.045186,0.010092,0.999423,None
